# Non-linear Panel Models
---
## Overview (and pre-analysis plan)
In this notebook, we will be:
- Fitting multiple panel models to non-linear data (i.e. data with binary outcomes)
- Models include:
    1. Pooled logit
    2. Conditional logit (country fixed effects)
    3. Conditional logit (year fixed effects)
    4. LPM
    5. LPM (country fixed effects)
    6. LPM (year fixed effects)
    7. LPM (two-way fixed effects)
- Looking at a quick and dirty example of model selection using:
    - R squared
    - Akaike Information Criterion
    - Bayesian Information Criterion
    
Note: we will once again use `statsmodels` for model fitting and model selection.

Further note: I've deliberately left out cross-validation for model selection. I believe this technique deserves its own separate notebook.

In [1]:
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.discrete.conditional_models import ConditionalLogit

# 1. Data prep

In [2]:
# Load data (from http://www.princeton.edu/~otorres/LogitR101.pdf )

data = pd.read_stata('http://dss.princeton.edu/training/Panel101.dta')
data

,country,year,y,y_bin,x1,x2,x3,opinion,op
0,A,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,Str agree,1.0
1,A,1991,-1.899661e+09,0.0,0.320685,-0.948720,0.492538,Disag,0.0
2,A,1992,-1.123436e+07,0.0,0.363466,-0.789484,0.702523,Disag,0.0
3,A,1993,2.645775e+09,1.0,0.246144,-0.885533,-0.094391,Disag,0.0
4,A,1994,3.008335e+09,1.0,0.424623,-0.729768,0.946131,Disag,0.0
...,...,...,...,...,...,...,...,...,...
65,G,1995,1.323696e+09,1.0,1.087186,-1.409817,2.829808,Str disag,0.0
66,G,1996,2.545242e+08,1.0,0.781075,-1.328000,4.278224,Str agree,1.0
67,G,1997,3.297033e+09,1.0,1.257879,-1.577367,4.587326,Disag,0.0
68,G,1998,3.011821e+09,1.0,1.242777,-1.601218,6.113762,Disag,0.0


In [24]:
# Seperate exogenous columns, endogenous columns, 
# country column, and year column

exog = data[['x1', 'x2', 'x3']]
endog = data['y_bin'].astype(int)
countries = data['country']
years = data['year']

# 2. Non-linear panel models

## 2.1 Pooled logit

In [4]:
# Pooled logit

logit_mod = smf.logit(formula='y_bin ~ x1 + x2 + x3',
                      data=data)
logit_res = logit_mod.fit()
logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.467944
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  y_bin   No. Observations:                   70
Model:                          Logit   Df Residuals:                       66
Method:                           MLE   Df Model:                            3
Date:                Wed, 10 Mar 2021   Pseudo R-squ.:                 0.06486
Time:                        21:06:42   Log-Likelihood:                -32.756
converged:                       True   LL-Null:                       -35.028
Covariance Type:            nonrobust   LLR p-value:                    0.2084
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4262      0.639      0.667      0.505      -0.826       1.679
x1             0.8618      0.784      1.099      0.272      -0.675       2.398
x2             0.3665      0.308      1.189      0.234      -0.237       0.971
x3             0.7512      0.455      1.652      0.099      -0.140       1.643
==============================================================================
"""

## 2.2 Conditional logit (country fixed effects)

In [12]:
# Get variance within each country for y_bin, x1, x2, and x3

within_country_var = data[['y_bin', 'x1', 'x2', 'x3', 'country']].groupby(by='country').var()
within_country_var

,y_bin,x1,x2,x3
country,,,,
A,0.177778,0.016893,0.019946,0.345907
B,0.277778,0.111110,0.014658,0.157842
C,0.233333,0.013766,0.004341,0.368436
D,0.100000,0.059227,0.030829,0.443879
E,0.100000,0.166607,0.114101,0.656200
F,0.100000,0.304088,0.029110,0.397462
G,0.100000,0.037319,0.012824,4.367728


In [30]:
# Conditional logit (country FE)
# Drop countries without within-group variance

clogit_country_mod = ConditionalLogit(endog=endog,
                                      exog=exog,
                                      groups=countries)
clogit_country_res = clogit_country_mod.fit(maxiter=1000)
clogit_country_res.summary()

/home/topher-lo/miniconda3/envs/ec333/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


ValueError: need covariance of parameters for computing (unnormalized) covariances

## 2.3 Conditional logit (year fixed effects)

In [17]:
# Get variance within each country for y_bin, x1, x2, and x3

within_year_var = data[['y_bin', 'x1', 'x2', 'x3', 'year']].groupby(by='year').var()
within_year_var

,y_bin,x1,x2,x3
year,,,,
1990,0.238095,0.456598,2.215550,0.290444
1991,0.285714,0.190625,2.501216,0.209760
1992,0.285714,0.230787,2.103001,0.522366
1993,0.000000,0.278362,2.172129,0.801000
1994,0.000000,0.150581,2.647259,1.173429
1995,0.142857,0.135641,1.794507,1.221629
1996,0.000000,0.117954,1.909766,2.957404
1997,0.000000,0.238063,2.155005,3.564464
1998,0.238095,0.137527,2.085431,5.010176


In [22]:
# Conditional logit (year FE)

clogit_country_mod = ConditionalLogit(endog=endog,
                                      exog=exog,
                                      groups=years)
clogit_country_res = clogit_country_mod.fit(maxiter=1000)
clogit_country_res.summary()

ValueError: need covariance of parameters for computing (unnormalized) covariances

## 2.4 LPM

In [ ]:
# Pooled LPM

